In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p2/train_data.csv
/kaggle/input/2022-ai-w3p2/test_data.csv
/kaggle/input/2022-ai-w3p2/sample_submit.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('../input/2022-ai-w3p2/train_data.csv')
test = pd.read_csv('../input/2022-ai-w3p2/test_data.csv')
submit = pd.read_csv('../input/2022-ai-w3p2/sample_submit.csv')

In [4]:
trainX = train.drop(['ID', 'price'], axis=1)
trainY = train['price']
testX = test.drop('ID', axis=1)

In [5]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17085 entries, 0 to 17084
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company       17085 non-null  int64  
 1   model         17085 non-null  object 
 2   year          17085 non-null  int64  
 3   transmission  17085 non-null  int64  
 4   mileage       17085 non-null  int64  
 5   fueltype      17085 non-null  int64  
 6   tax           17085 non-null  int64  
 7   mpg           17085 non-null  float64
 8   engineSize    17085 non-null  float64
dtypes: float64(2), int64(6), object(1)
memory usage: 1.2+ MB


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

trainX['model'] = le.fit_transform(trainX['model'])
testX['model'] = le.transform(testX['model'])

In [7]:
x_train = torch.FloatTensor(trainX.to_numpy())
y_train = torch.FloatTensor(trainY.to_numpy().reshape(-1,1))
x_test = torch.FloatTensor(testX.to_numpy())

In [8]:
x_train.shape[1]

9

In [9]:
# 초기화
W = torch.zeros([x_train.shape[1], 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [10]:
# optimizer 설정
import torch.optim as optim
optimizer = optim.SGD([W, b], 0.00001)

In [11]:
np_epochs = 1000000
for epoch in range(np_epochs+1) :
    # H(x) 계산
    hypothesis = torch.matmul(x_train, W) +b
    # cost 계산 - 
    cost = torch.mean(torch.abs(hypothesis - y_train))
     
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100000 == 0 :
        print(epoch, cost.item())

0 16819.763671875
100000 6059.2265625
200000 5972.69287109375
300000 5943.283203125
400000 5932.3359375
500000 5927.30517578125
600000 5924.2451171875
700000 5921.87841796875
800000 5919.6923828125
900000 5917.595703125
1000000 5915.5546875


In [12]:
# 내부적으로 계산하는 require_grad 때문에 상수화가 안돼서? detach로 함수를 제거한다 (?)
submit['price'] = (torch.matmul(x_test, W) + b).detach()

In [13]:
submit.to_csv('submit2.csv', index=False)